Imports

In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd
from RealtimeTracking import *
import gc
import easyocr
import numpy as np 
import csv
from scipy.interpolate import interp1d


from LicensePlate import LicensePlateDetection, LicensePlateReader
from Vehicle import VehicleDetection
from utils import PlateToVehicle, interpolate_bounding_boxes

In [2]:
gc.collect()

0

Load the model

In [3]:
plate_det_model_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/models/YOLOv8/yolov8n_90e_cust/runs/detect/train4/weights/best.pt'
testvideo_path = '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/testvideo_highway_mp4.mp4'

In [4]:
#visualization
import cv2
import pandas as pd
import ast

def draw_bounding_boxes(input_video_path, csv_file_path, output_video_path):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file_path)

    # Open the input video
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise Exception(f"Failed to open video file: {input_video_path}")

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define the codec and create a VideoWriter object for the output video
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")  # You can change the codec as needed
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Process each frame and draw bounding boxes
    frame_number = 0
    ret=True
    while ret:
        ret, frame = cap.read()

        # Filter the DataFrame for the current frame number
        frame_data = df[df['fr_number'] == frame_number]        

        #print(frame_data)
        for _, row in frame_data.iterrows():
            #car
            car_bbox_vals = row['v_bbox'].split(' ')
            car_bbox_vals[:]=map(lambda x: int(float(x)),car_bbox_vals)
            car_bbox_vals = str(car_bbox_vals)
            car_bbox_vals = "[" + car_bbox_vals[1:-1] + "]"
            car_bbox = ast.literal_eval(car_bbox_vals)


            #license plate
            lp_bbox_vals = row['lp_bbox'].split(' ')
            lp_bbox_vals[:]=map(lambda x: int(float(x)),lp_bbox_vals)
            lp_bbox_vals = str(lp_bbox_vals)
            lp_bbox_vals_str = "[" + lp_bbox_vals[1:-1] + "]"
            license_plate_bbox = ast.literal_eval(lp_bbox_vals_str)

            # Draw bounding boxes on the frame
            #cv2.rectangle(frame, (int(car_bbox[0]), int(car_bbox[1]), (int(car_bbox[2]), int(car_bbox[3]))), (0, 255, 0), 2)
            cv2.rectangle(frame, (int(car_bbox[0]), int(car_bbox[1])), (int(car_bbox[2]), int(car_bbox[3])), (0, 255, 0), 1,cv2.LINE_4)
            cv2.rectangle(frame, (int(license_plate_bbox[0]), int(license_plate_bbox[1])),
                          (int(license_plate_bbox[2]), int(license_plate_bbox[3])), (0, 0, 255), 1)

            lp_bbox_vals = row['lp_bbox'].split(' ')
            lp_bbox_vals[:]=map(lambda x: int(float(x)),lp_bbox_vals)
            car_bbox_vals = row['v_bbox'].split(' ')
            car_bbox_vals[:]=map(lambda x: int(float(x)),car_bbox_vals)
            #print(lp_bbox_vals)
            #x1, y1, x2, y2
            #H, W, _ = lp_bbox_vals.shape
            H = lp_bbox_vals[3]-lp_bbox_vals[1]
            W = lp_bbox_vals[2]-lp_bbox_vals[0]
            #print(H,W)

            try:
                #frame[int(car_bbox_vals[1]) - H - 100:int(car_bbox_vals[1]) - 100,
                #      int((car_bbox_vals[2] + car_bbox_vals[0] - W) / 2):int((car_bbox_vals[2] + car_bbox_vals[0] + W) / 2), :] = license_crop

                #frame[int(car_bbox_vals[1]) - H - 400:int(car_bbox_vals[1]) - H - 100,
                #      int((car_bbox_vals[2] + car_bbox_vals[0] - W) / 2):int((car_bbox_vals[2] + car_bbox_vals[0] + W) / 2), :] = (255, 255, 255)

                license_crop = frame[lp_bbox_vals[1]+2:lp_bbox_vals[3],lp_bbox_vals[0]+2:lp_bbox_vals[2], :]
                #frame[int(car_bbox_vals[1]) - H - 100:int(car_bbox_vals[1]) - 100,
                #      int((car_bbox_vals[2] + car_bbox_vals[0] - W) / 2):int((car_bbox_vals[2] + car_bbox_vals[0] + W) / 2), :] = license_crop
                #cv2.imshow('frame', license_crop)
                #cv2.waitKey(0)

                padding = 10
                border_thickness = 1
                (text_width, text_height), _ = cv2.getTextSize(
                    row['lp'],
                    cv2.FONT_HERSHEY_SIMPLEX,
                    2,
                    5)
                text_x = int((car_bbox[0] + car_bbox[2] - text_width) / 2)
                text_y = int(car_bbox[1] - H + (text_height / 2))

                background_width = car_bbox[2] - car_bbox[0]
                cv2.rectangle(frame, (car_bbox[0], car_bbox[1]-text_height-H), (car_bbox[2], car_bbox[1]), (0, 0, 0), thickness=cv2.FILLED)
                #cv2.rectangle(frame, (text_x, text_y - text_height), (text_x + background_width, text_y), (255, 255, 255), thickness=cv2.FILLED)

                #cv2.rectangle(frame, (text_x-padding, text_y-text_height-padding), (text_x + text_width+padding, text_y+padding), (0, 0, 0), thickness=cv2.FILLED)
                cv2.rectangle(frame, (car_bbox[0], car_bbox[1]-text_height-H), (car_bbox[2], car_bbox[1]), (0, 255, 0), thickness=border_thickness)

                #frame[car_bbox_vals[1]+2:car_bbox_vals[3], car_bbox_vals[0]+2:car_bbox_vals[2], :] = license_crop
                #frame[car_bbox[1]:car_bbox[1]+(lp_bbox_vals[3]-lp_bbox_vals[1]), car_bbox[0]:car_bbox[0]+(lp_bbox_vals[2]-lp_bbox_vals[0]), :] = license_crop
                #x=100
                #y=100
                #alpha = 1.0  # Controls the transparency of the cropped image
                #beta = 1.0 - alpha
                #cropped_height = lp_bbox_vals[3] - lp_bbox_vals[1] - 2
                #cropped_width = lp_bbox_vals[2] - lp_bbox_vals[0] - 2

                #cv2.addWeighted(frame, alpha, license_crop, beta, 0, frame[y:y + cropped_height, x:x + cropped_width])

                if(row['lp'] != '0'):
                    cv2.putText(frame,
                            row['lp'],
                            (int((car_bbox[0] + car_bbox[2] - text_width) / 2), int(car_bbox[1] - H + (text_height / 2))),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            2,
                            (255, 255, 255),
                            3)

            except:
                pass

        # Write the frame with bounding boxes to the output video
        #try:
        if frame is not None:
            out.write(frame)
        #except:
        #    pass
        frame_number += 1
        #frame = cv2.resize(frame, (1280, 720))

        #cv2.imshow('frame', frame)
        #cv2.waitKey(0)

    # Release the video capture and writer objects
    cap.release()
    out.release()

Capture on a video

In [5]:
# OOP
df_cols = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
df_rows = []
# load using Sort
video_capture = cv2.VideoCapture(testvideo_path)

vehicle_detection = VehicleDetection('YOLOv8')
lp_detection = LicensePlateDetection('yolov8n_90e_cust')
lp_reader = LicensePlateReader('easyocr')

mot_trckr = Sort()
#results = {}
ret = True
frame_indexer=0
while ret:
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
    
    ret, frame = video_capture.read()
    
    if ret:
        #if frame_indexer>10:
        #    break
        #results[frame_indexer] = {}
        #det
        det_objs = vehicle_detection.Detect(frame)
        vehicles=[]
        for det_obj in det_objs.boxes.data.tolist():
            obj_x1, obj_y1, obj_x2, obj_y2, obj_score, obj_class_id = det_obj
            if int(obj_class_id) in vehicle_detection.vehicle_ids:
                vehicles.append([obj_x1, obj_y1, obj_x2, obj_y2, obj_score])
                #print(obj_class_id)
        
        track_ids = mot_trckr.update(np.asarray(vehicles))
        #print(track_ids)
        #plates=[]
        det_lps = lp_detection.Detect(frame)
        for det_lp in det_lps.boxes.data.tolist():
            lp_x1, lp_y1, lp_x2, lp_y2, lp_score, lp_class_id = det_lp
            
            #assign license plate to vehicle
            vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2,vehicle_id = PlateToVehicle(det_lp,track_ids)
            if vehicle_id != -1:
                #crop
                license_plate = frame[int(lp_y1):int(lp_y2),int(lp_x1):int(lp_x2),:]
                lp_text, lp_text_confscore, lp_prep = lp_reader.ModifiedPrepAndRead(license_plate)

                #cv2.imshow('lp',license_plate)
                #cv2.waitKey(0)

                #cv2.imshow('lp',lp_psrep)
                #print(lp_text, lp_text_confscore)
                #cv2.waitKey(0)

                if lp_text != None:
                    df_rows.append({'fr_number':frame_indexer,
                                    'v_id':int(vehicle_id+1),
                                    'v_bbox':[vehicle_x1,vehicle_y1,vehicle_x2,vehicle_y2],
                                    'lp_bbox':[lp_x1, lp_y1, lp_x2, lp_y2],
                                    'lp_bbox_score':lp_score,
                                    'lp':lp_text,
                                    'lp_score':lp_text_confscore})
                
            
    
    frame_indexer+=1

[ WARN:0@2.748] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@2.748] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin0 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@2.748] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@2.749] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_16645483

In [6]:
out = pd.DataFrame(df_rows,columns=df_cols)
out.to_csv('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/logs/log.csv')

In [7]:
# Load the CSV file
with open('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/logs/log.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Interpolate missing data
interpolated_data = interpolate_bounding_boxes(data,True)

# Write updated data to a new CSV file
header = ['fr_number','v_id', 'v_bbox','lp_bbox','lp_bbox_score','lp','lp_score']
with open('/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/logs/log_interpolated.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(interpolated_data)

In [8]:
draw_bounding_boxes(testvideo_path,
                    '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/logs/log_interpolated.csv',
                    '/Users/banoczymartin/Library/Mobile Documents/com~apple~CloudDocs/OE/platedetector/video_data/out.avi')


[ WARN:0@561.973] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@561.973] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin1 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@561.974] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@561.974] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_